In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from tqdm.notebook import trange, tqdm

os.chdir("..")
from src.information_bottleneck import IB
from src.config import Ns, betas
from src.utils import BETA_STR

### P(X=x, Y=y) in the Experiment Setup
Assume apriory unfirom distribution to the oddball chance, these are the probabilies for the joint distribution of X and Y:\
P(Yt+1 = 0, Xt = n) = (N-n+1) / ((N+2)(N+1))

![image.png](images/py1X.png)

In [2]:
def calculate_pXY(N):
    pXY0 = [N-n+1 for n in range(N+1)]
    pXY1 = [n+1 for n in range(N+1)]
    denominator = (N+1)*(N+2)
    pXY = np.column_stack([pXY0,pXY1])/denominator
    return pXY

def init_pXhat_X(N):
    return np.eye(N+1)

out_vals = ["p(Xhat|X)","p(Y|Xhat)","I(Xhat;X)","I(Xhat;Y)"]
d_results = {N:dict(zip(np.arange(len(betas)), [None]*len(betas))) for N in Ns}

In [12]:
pd.read_pickle("data/df_surprise.pkl").applymap(lambda x: x.shape)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
1,"(2, 2)","(2, 2)","(2, 2)","(2, 2)","(2, 2)","(2, 2)","(2, 2)","(2, 2)","(2, 2)","(2, 2)","(2, 2)","(2, 2)","(2, 2)","(2, 2)","(2, 2)","(2, 2)","(2, 2)","(2, 2)","(2, 2)","(2, 2)"
2,"(2, 3)","(2, 3)","(2, 3)","(2, 3)","(2, 3)","(2, 3)","(2, 3)","(2, 3)","(2, 3)","(2, 3)","(2, 3)","(2, 3)","(2, 3)","(2, 3)","(2, 3)","(2, 3)","(2, 3)","(2, 3)","(2, 3)","(2, 3)"
3,"(2, 4)","(2, 4)","(2, 4)","(2, 4)","(2, 4)","(2, 4)","(2, 4)","(2, 4)","(2, 4)","(2, 4)","(2, 4)","(2, 4)","(2, 4)","(2, 4)","(2, 4)","(2, 4)","(2, 4)","(2, 4)","(2, 4)","(2, 4)"
4,"(2, 5)","(2, 5)","(2, 5)","(2, 5)","(2, 5)","(2, 5)","(2, 5)","(2, 5)","(2, 5)","(2, 5)","(2, 5)","(2, 5)","(2, 5)","(2, 5)","(2, 5)","(2, 5)","(2, 5)","(2, 5)","(2, 5)","(2, 5)"
5,"(2, 6)","(2, 6)","(2, 6)","(2, 6)","(2, 6)","(2, 6)","(2, 6)","(2, 6)","(2, 6)","(2, 6)","(2, 6)","(2, 6)","(2, 6)","(2, 6)","(2, 6)","(2, 6)","(2, 6)","(2, 6)","(2, 6)","(2, 6)"
6,"(2, 7)","(2, 7)","(2, 7)","(2, 7)","(2, 7)","(2, 7)","(2, 7)","(2, 7)","(2, 7)","(2, 7)","(2, 7)","(2, 7)","(2, 7)","(2, 7)","(2, 7)","(2, 7)","(2, 7)","(2, 7)","(2, 7)","(2, 7)"
7,"(2, 8)","(2, 8)","(2, 8)","(2, 8)","(2, 8)","(2, 8)","(2, 8)","(2, 8)","(2, 8)","(2, 8)","(2, 8)","(2, 8)","(2, 8)","(2, 8)","(2, 8)","(2, 8)","(2, 8)","(2, 8)","(2, 8)","(2, 8)"
8,"(2, 9)","(2, 9)","(2, 9)","(2, 9)","(2, 9)","(2, 9)","(2, 9)","(2, 9)","(2, 9)","(2, 9)","(2, 9)","(2, 9)","(2, 9)","(2, 9)","(2, 9)","(2, 9)","(2, 9)","(2, 9)","(2, 9)","(2, 9)"
9,"(2, 10)","(2, 10)","(2, 10)","(2, 10)","(2, 10)","(2, 10)","(2, 10)","(2, 10)","(2, 10)","(2, 10)","(2, 10)","(2, 10)","(2, 10)","(2, 10)","(2, 10)","(2, 10)","(2, 10)","(2, 10)","(2, 10)","(2, 10)"
10,"(2, 11)","(2, 11)","(2, 11)","(2, 11)","(2, 11)","(2, 11)","(2, 11)","(2, 11)","(2, 11)","(2, 11)","(2, 11)","(2, 11)","(2, 11)","(2, 11)","(2, 11)","(2, 11)","(2, 11)","(2, 11)","(2, 11)","(2, 11)"


### Calculate Results of IB method and Model Surprise Predictor

![image.png](images/SIB.png)

In [3]:
for N in tqdm(d_results.keys(), desc="N"):
#     time_print("-"*10+f" N = {N} "+"-"*10)
    for beta_ind in tqdm(d_results[N].keys(), leave=False, desc=BETA_STR):
        # calculate IB algorithm
        d_results[N][beta_ind] = IB(
            pXY=calculate_pXY(N),
            beta=betas[beta_ind],
            p0Xhat_X=init_pXhat_X(N)
        )
        
        # store results
        pXhat_X = d_results[N][beta_ind]["p(Xhat|X)"]
        pY_Xhat = d_results[N][beta_ind]["p(Y|Xhat)"]
        d_results[N][beta_ind]["surprise"] = -np.log2(pY_Xhat) @ pXhat_X

N:   0%|          | 0/50 [00:00<?, ?it/s]

β:   0%|          | 0/20 [00:00<?, ?it/s]

β:   0%|          | 0/20 [00:00<?, ?it/s]

β:   0%|          | 0/20 [00:00<?, ?it/s]

β:   0%|          | 0/20 [00:00<?, ?it/s]

...

β:   0%|          | 0/20 [00:00<?, ?it/s]

β:   0%|          | 0/20 [00:00<?, ?it/s]

β:   0%|          | 0/20 [00:00<?, ?it/s]

β:   0%|          | 0/20 [00:00<?, ?it/s]

β:   0%|          | 0/20 [00:00<?, ?it/s]

In [10]:
# unpacking
df_results = pd.DataFrame.from_dict(d_results, orient='index')
df_mi_compression = df_results.applymap(lambda d: d["I(Xhat;X)"])
df_mi_accuracy = df_results.applymap(lambda d: d["I(Xhat;Y)"])
df_pXhat_X = df_results.applymap(lambda d: d["p(Xhat|X)"])
df_pY_Xhat = df_results.applymap(lambda d: d["p(Y|Xhat)"])
df_surprise = df_results.applymap(lambda d: d["surprise"])

# saving
df_mi_compression.to_pickle("data/df_mi_compression.pkl")
df_mi_accuracy.to_pickle("data/df_mi_accuracy.pkl")
df_pXhat_X.to_pickle("data/df_pXhat_X.pkl")
df_pY_Xhat.to_pickle("data/df_pY_Xhat.pkl")
df_surprise.to_pickle("data/df_surprise.pkl")